In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import xml.etree.ElementTree as ET 

In [ ]:
path_an = "/content/drive/MyDrive/Vehicle-number-plate-recognition-YOLOv5-010f57b81b8fdde8aa18e52d6ad29b3109178180/annotations"

In [ ]:
dataset = {
            "xmin":[],
            "ymin":[],   
            "xmax":[],
            "ymax":[],
            "name":[],    
            "file":[],
            "width":[],
            "height":[],
           }

for anno in glob.glob(path_an+"/*.xml"):
    tree = ET.parse(anno)
    
    for elem in tree.iter():
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag: 
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))    

        if 'object' in elem.tag:
            for attr in list(elem):
                
                if 'name' in attr.tag:
                    name = attr.text                 
                    dataset['name']+=[name]
                    dataset['width']+=[width]
                    dataset['height']+=[height] 
                    dataset['file']+=[anno.split('/')[-1][0:-4]] 
                            
                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin']+=[xmin]
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin']+=[ymin]                                
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax']+=[xmax]                                
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax']+=[ymax]                        
                

In [ ]:
from google.colab import data_table
from vega_datasets import data

data0=pd.DataFrame(dataset)
data0

,xmin,ymin,xmax,ymax,name,file,width,height
0,7,16,493,256,licence,Cars114,500,300
1,175,114,214,131,licence,Cars100,400,267
2,230,129,248,134,licence,Cars103,400,196
3,189,116,202,121,licence,Cars103,400,196
4,319,165,478,204,licence,Cars111,600,338
...,...,...,...,...,...,...,...,...
466,98,102,297,191,licence,Cars97,400,300
467,85,196,247,259,licence,Cars98,370,400
468,133,126,261,160,licence,Cars96,400,248
469,281,185,393,271,licence,Cars92,600,400


In [ ]:
print(data0['name'].unique())

['licence']


In [ ]:
data0['class']=data0['name'].map({'licence':0})
data0['Xcent']=(data0['xmin']+data0['xmax'])/(2*data0['width'])
data0['Ycent']=(data0['ymin']+data0['ymax'])/(2*data0['height'])
data0['boxW']=(data0['xmax']-data0['xmin'])/data0['width']
data0['boxH']=(data0['ymax']-data0['ymin'])/data0['height']

In [ ]:
data0

,xmin,ymin,xmax,ymax,name,file,width,height,class,Xcent,Ycent,boxW,boxH
0,7,16,493,256,licence,Cars114,500,300,0,0.500000,0.453333,0.972000,0.800000
1,175,114,214,131,licence,Cars100,400,267,0,0.486250,0.458801,0.097500,0.063670
2,230,129,248,134,licence,Cars103,400,196,0,0.597500,0.670918,0.045000,0.025510
3,189,116,202,121,licence,Cars103,400,196,0,0.488750,0.604592,0.032500,0.025510
4,319,165,478,204,licence,Cars111,600,338,0,0.664167,0.545858,0.265000,0.115385
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,98,102,297,191,licence,Cars97,400,300,0,0.493750,0.488333,0.497500,0.296667
467,85,196,247,259,licence,Cars98,370,400,0,0.448649,0.568750,0.437838,0.157500
468,133,126,261,160,licence,Cars96,400,248,0,0.492500,0.576613,0.320000,0.137097
469,281,185,393,271,licence,Cars92,600,400,0,0.561667,0.570000,0.186667,0.215000


In [ ]:
data0.to_csv('annotation.csv',index=False)-